# Advent Of Code 2023

## Day 1: Trebuchet?!

In [ ]:
# Part 1
def extract_number(from_line: str) -> int:
    clean_line = ''.join(char for char in from_line if char.isdigit())
    return int(clean_line[0] + clean_line[-1])

with open('files/day1-input', 'r', encoding='utf-8') as input_file:
    print(f"The sum of all the calibration values is {sum(extract_number(line) for line in input_file)}")

In [ ]:
# Part 2
DIGIT_REPLACEMENT = {
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5,
    'six': 6,
    'seven': 7,
    'eight': 8,
    'nine': 9
}

def extract_number(from_line: str) -> int:
    
    first_digit, last_digit = None, None

    for (index, char) in enumerate(from_line):
        
        # Match regular digit
        if char.isdigit():
            last_digit = char
        
        else:
            # Match spelled out digit
            matches = [value for digit, value in DIGIT_REPLACEMENT.items() if from_line[index:].startswith(digit)]
            if any(matches):
                last_digit = str(matches[0])
    
        if first_digit is None:
            first_digit = last_digit
            
    return int(first_digit + last_digit)

with open('files/day1-input', 'r', encoding='utf-8') as input_file:
    print(f"The sum of all the calibration values is {sum(extract_number(line) for line in input_file)}")

## Day 2 - Cube conundrum

In [ ]:
# Part 1
import re

MAX_AMOUNTS = {
    "red":   12,
    "green": 13,
    "blue":  14,
}

RE_PARSE_GAME = re.compile(r"Game (\d+): (.*)")
RE_PARSE_DRAW = re.compile(r"(\d+) (red|green|blue)")

def process_game(line: str) -> tuple[int, int]:
    """Process the game result given as a string
    Returns a tuple containing the id of the game and True if the game is possible.
    """
    game_id, game_content = RE_PARSE_GAME.match(line).groups()

    return int(game_id), all(
        int(number) <= MAX_AMOUNTS[color]
        for number, color in (
            RE_PARSE_DRAW.search(bit).groups()
            for draw in game_content.split(sep=";")
            for bit in draw.split(sep=",")
        )
    )

with open('files/day2-input', 'r', encoding='utf-8') as input_file:
    print(f"The sum of all possible game IDs is {sum(id for id, result in (process_game(line) for line in input_file) if result)}")

In [ ]:
# Part 2
import re
import operator
from functools import reduce

RE_PARSE_GAME = re.compile(r"Game (\d+): (.*)")
RE_PARSE_DRAW = re.compile(r"(\d+) (red|green|blue)")

def process_game_power(line: str) -> int:
    """Process the game result given as a string
    Returns the power of the minimal possible set of cubes for the game.
    """
    game_id, game_content = RE_PARSE_GAME.match(line).groups()

    min_amounts = dict()
    for number, color in (RE_PARSE_DRAW.search(bit).groups() for draw in game_content.split(sep=";") for bit in draw.split(sep=",")):
        min_amounts[color] = max(int(number), min_amounts.get(color, 0))
        
    return reduce(operator.mul, min_amounts.values(), 1)

with open('files/day2-input', 'r', encoding='utf-8') as input_file:
    print(f"The sum of all possible game IDs is {sum(process_game_power(line) for line in input_file)}")

## Day 3 - Gear ratios

In [ ]:
# Part 1
import re

RE_FIND_NUMBER = re.compile(r"\d+")

with open('files/day3-input', encoding='utf-8') as input_file:
    engine: tuple[str, ...] = tuple(line.strip() for line in input_file)

# Compute boundaries
ENGINE_HEIGHT = len(engine)
ENGINE_WIDTH = len(engine[0])

total = 0

for row, line in enumerate(engine):
    for match in RE_FIND_NUMBER.finditer(line):
        if any(
            char != "." and not char.isdigit()
            for subline in engine[max(0, row - 1) : min(ENGINE_HEIGHT, row + 2)]
            for char in subline[max(0, match.start() - 1) : min(ENGINE_WIDTH, match.end() + 1)]
        ):
            total += int(match.group(0))

print(f"The sum of all part number is {total}")

In [1]:
# Part 2
import re

RE_FIND_NUMBER = re.compile(r"\d+")

with open('files/day3-input', encoding='utf-8') as input_file:
    engine: tuple[str, ...] = tuple(line.strip() for line in input_file)

# Compute boundaries
ENGINE_HEIGHT = len(engine)
ENGINE_WIDTH = len(engine[0])

# This dictionary uses as key the position of a gear, and holds as value the list of part number adjacent to that gear
gears: dict[tuple[int, int], list[int]] = dict()

# Check every number for adjacent gear ; add it to gears dictionary
for row, line in enumerate(engine):
    for match in RE_FIND_NUMBER.finditer(line):
        for i in range(max(0, row - 1), min(ENGINE_HEIGHT, row + 2)):
            for j in range(max(0, match.start() - 1), min(ENGINE_WIDTH, match.end() + 1)):
                if engine[i][j] == "*":
                    gears[(i, j)] = gears.get((i, j), []) + [int(match.group(0))]


print(f"The sum of all part number is {sum(parts[0] * parts[1] for parts in gears.values() if len(parts) == 2)}")

The sum of all part number is 75741499


## Day 4 - Scratchcards

In [23]:
# Part 1
import re

RE_SCRATCHCARD = re.compile(r"Card\s+(\d+):\s*((?:\d+\s*)*) \| ((?:\s*\d+)*)\s*")

def compute_scratchcard(card: str) -> int:
    """Computes the score of a scratchcard given its string representation"""
    card_id, winning_numbers, your_numbers = RE_SCRATCHCARD.match(card).groups()
    return int(2**(sum(1 for number in your_numbers.split() if number and number in winning_numbers.split()) - 1))
    
with open('files/day4-input', encoding='utf-8') as input_file:
    print(f"The sum of all the scratchcards score is {sum(compute_scratchcard(line) for line in input_file)}")
    

The sum of all the scratchcards score is 21959


In [40]:
# Part 2
import re

RE_SCRATCHCARD = re.compile(r"Card\s+(\d+):\s*((?:\d+\s*)*) \| ((?:\s*\d+)*)\s*")

# This dictionary stores the number of each scratchcard (<card ID>: <number of cards>)
count_scratchcards: dict[int, int] = dict()

def compute_scratchcard(card: str) -> None:
    """Computes the scratchcards won by a scratchcard given its string representation"""
    card_id, winning_numbers, your_numbers = RE_SCRATCHCARD.match(card).groups()
    card_id = int(card_id)
    
    # Compute score
    score = sum(1 for number in your_numbers.split() if number and number in winning_numbers.split())
    
    # Update counts
    count_scratchcards[card_id] = count_scratchcards.get(card_id, 0) + 1
    for i in range(card_id + 1, card_id + score + 1):
        count_scratchcards[i] = count_scratchcards.get(i, 0) + count_scratchcards[card_id]
    
with open('files/day4-input', encoding='utf-8') as input_file:
    [compute_scratchcard(line) for line in input_file]
    print(f"The sum of all the scratchcards score is {sum(count for count in count_scratchcards.values())}")
    

The sum of all the scratchcards score is 5132675
